In [13]:
import json
import csv
from datetime import datetime
import time
import os
import requests
import re
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import threading
import random
import shutil


In [12]:
def scrape(sale_id):
    path = f"https://catalogue.astonbarclay.net/vehicle-search/?search=true&sites=1&sales={sale_id}"

    options = ChromeOptions()
    options.headless = False
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(path)
    driver.maximize_window()

    # ✅ Accept cookie popup
    try:
        cookie = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
        )
        cookie.click()
    except:
        pass

    # ✅ Sync cookies into session
    cookies = driver.get_cookies()
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    # ✅ Check login status
    already_logged_in = False
    try:
        resp = session.get("https://catalogue.astonbarclay.net/api/user")
        if resp.status_code == 200:
            user = resp.json()
            if user.get("loggedIn"):
                already_logged_in = True
                print(f"🔒 Already logged in as {user.get('name')}")
    except:
        pass

    # ✅ Login if required
    if not already_logged_in:
        print("🔑 Logging in...")
        try:
            login_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".acc_button_alt"))
            )
            login_btn.click()

            user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "Username")))
            user_name.send_keys("sultanmirza0501@gmail.com")

            password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "Password")))
            password.send_keys("Muhssan7865")

            submit_btn = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Login']")
            submit_btn.click()

            print("✅ Logged in successfully.")
            time.sleep(5)
        except Exception as e:
            print(f"❌ Login step failed: {e}")

    # ✅ Fetch vehicle IDs
    cookies = driver.get_cookies()
    session.cookies.clear()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    api_url = f"https://catalogue.astonbarclay.net/api/vehicle/all?siteid=1&saleid={sale_id}"
    response = session.get(api_url)
    if response.status_code == 200:
        data = response.json()
        ids = [item.get("id") for item in data.get("result", []) if "id" in item]

        filename = "aston_ids.json"
        if os.path.exists(filename):
            base_folder = "json"
            if os.path.exists(base_folder):
                random_suffix = str(random.randint(1000, 9999))
                base_folder = f"{base_folder}_{random_suffix}"

            os.makedirs(base_folder, exist_ok=True)
            filename = os.path.join(base_folder, "aston_ids.json")
        with open(filename, "w", encoding="utf-8") as f:
            json.dump({"count": len(ids), "ids": ids}, f, indent=4)

        print(f"✅ {len(ids)} IDs saved to '{filename}'")

    else:
        print(f"❌ Failed to fetch IDs — status {response.status_code}")

   
    return driver


def fetch_vehicle_details(driver):
    ids_file = "aston_ids.json"
    output_file = "aston_vehicles.json"

    if not os.path.exists(ids_file):
        print("❌ IDs file not found!")
        return

    with open(ids_file, "r", encoding="utf-8") as f:
        ids = json.load(f).get("ids", [])

    if not ids:
        print("❌ No IDs found.")
        return

    print(f"📦 Fetching {len(ids)} vehicle details...")

    all_vehicles = []

    for vid in tqdm(ids, desc="Fetching vehicles"):
        try:
            driver.execute_script(f"window.open('https://catalogue.astonbarclay.net/api/vehicle/details/{vid}?view=catalogue', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(3)
            pre = driver.find_element(By.TAG_NAME, "pre")
            json_text = pre.text.strip()
            vehicle_data = json.loads(json_text)
            all_vehicles.append(vehicle_data)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"⚠️ Error fetching {vid}: {e}")
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_vehicles, f, indent=4)

    print(f"✅ Saved {len(all_vehicles)} vehicles in '{output_file}'")


    driver.quit()



driver = scrape(17580)
fetch_vehicle_details(driver)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=141.0.7390.123)
Stacktrace:
	GetHandleVerifier [0x0xaefe43+66515]
	GetHandleVerifier [0x0xaefe84+66580]
	(No symbol) [0x0x8ddc48]
	(No symbol) [0x0x8bc18d]
	(No symbol) [0x0x951a4e]
	(No symbol) [0x0x96c4d9]
	(No symbol) [0x0x94afc6]
	(No symbol) [0x0x91c2ca]
	(No symbol) [0x0x91d154]
	GetHandleVerifier [0x0xd47353+2521315]
	GetHandleVerifier [0x0xd422d3+2500707]
	GetHandleVerifier [0x0xb17c94+229924]
	GetHandleVerifier [0x0xb081f8+165768]
	GetHandleVerifier [0x0xb0ecad+193085]
	GetHandleVerifier [0x0xaf8158+100072]
	GetHandleVerifier [0x0xaf82f0+100480]
	GetHandleVerifier [0x0xae25aa+11066]
	BaseThreadInitThunk [0x0x75b37ba9+25]
	RtlInitializeExceptionChain [0x0x7726c3ab+107]
	RtlClearBits [0x0x7726c32f+191]


In [16]:
input_file = "aston_vehicles.json"
output_file = "barclay_renamed.csv"

headers = [
    "Title", "Derivative", "Images", "Auction Availability", "Lot", "Start Time", "Start Date",
    "Center", "Auction ID", "D.O.R", "Reg", "Mileage", "Mileage warranted", "Colour", "Body type",
    "Doors", "Fuel Type", "Transmission", "V5", "MOT Expiry Date", "CO2 emissions", "VAT status",
    "Former keepers", "Non Runner", "Vendor", "Service History", "Service Notes", "No of Service",
    "Last Service", "Last service mileage", "Grade", "CAP clean", "Euro Status", "CAP retail",
    "CAP average", "CAP below", "Autotrader Retail Value", "Autotrader Trade Value", "Powered by",
    "Inspection date", "VIN", "Exterior grade", "Interior grade", "Make", "Model", "CC",
    "Engine number", "Body plan", "Gearbox", "Paint type", "Colour Trim", "Seats",
    "Date of first reg", "Keys", "Inspection Report", "Other Report", "Tyres", "Spare Tyre",
    "Equipment", "Damaged_images", "Damage_details", "Year"
]

# ✅ Helper functions
def safe_list(val):
    if isinstance(val, (list, tuple)):
        return list(val)
    elif isinstance(val, str):
        return [val]
    return []

def safe_dict(val):
    return val if isinstance(val, dict) else {}

def safe_str(val):
    return str(val) if val is not None else ""

def format_datetime(dt_str):
    if not dt_str:
        return ""
    try:
        dt = datetime.fromisoformat(str(dt_str).replace("Z", "+00:00"))
        return dt.strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return ""

# ✅ Load JSON
with open(input_file, "r", encoding="utf-8") as f:
    vehicles = json.load(f)

# ✅ Write CSV
with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()

    for v in vehicles:
        v = safe_dict(v)
        inspection = safe_dict(v.get("inspection"))
        tyres = safe_dict(inspection.get("tyres"))
        vat_status = safe_dict(v.get("vatStatus"))
        site = safe_dict(v.get("site"))

        # ✅ Safe service extraction
        service_str = safe_str(v.get("serviceHistory"))
        entries = re.findall(
            r"(?:Service\s*(?:book|digital|invoice|print)?\s*)?(\d{2}/\d{2})\s*-\s*([\d,.Kk]+)",
            service_str
        )

        # ✅ Safe Start Date
        start_date_str = safe_str(v.get("saleStart"))

        # ✅ Build row
        row = {
            "Title": f"{safe_str(v.get('make'))} {safe_str(v.get('model'))}".strip(),
            "Derivative": safe_str(v.get("derivative")),
            "Images": ", ".join(safe_list(v.get("images"))),
            "Auction Availability": "Available" if not v.get("virtual", False) else "Virtual",
            "Lot": safe_str(v.get("lot")),
            "Start Time": (
                datetime.fromisoformat(start_date_str.replace("Z", "+00:00")).strftime("%H:%M")
                if start_date_str else ""
            ),
            "Start Date": format_datetime(start_date_str)[:10],
            "Center": safe_str(site.get("name")),
            "Auction ID": safe_str(v.get("saleId")),
            "D.O.R": safe_str(v.get("firstRegistration")),
            "Reg": safe_str(v.get("registration")),
            "Mileage": safe_str(v.get("mileage")),
            "Mileage warranted": safe_str(v.get("mileageWarranty")),
            "Colour": safe_str(v.get("colour")),
            "Body type": safe_str(v.get("bodyType")),
            "Doors": safe_str(v.get("doors")),
            "Fuel Type": safe_str(v.get("fuel")),
            "Transmission": safe_str(v.get("transmission")),
            "V5": safe_str(v.get("v5")),
            "MOT Expiry Date": safe_str(v.get("motExpiry")),
            "CO2 emissions": safe_str(v.get("co2")),
            "VAT status": safe_str(vat_status.get("label")),
            "Former keepers": safe_str(v.get("formerKeepers")),
            "Non Runner": safe_str(v.get("nonRunner")),
            "Vendor": safe_str(v.get("seller")),
            "Service History": service_str,
            "Service Notes": safe_str(v.get("remarks")),
            "No of Service": len(entries) if entries else "",
            "Last Service": entries[-1][0] if entries else "",
            "Last service mileage": entries[-1][1] if entries else "",
            "Grade": safe_str(v.get("namaGrade")),
            "CAP clean": safe_str(v.get("capCleanPrice")),
            "Euro Status": safe_str(v.get("eurostatus")),
            "CAP retail": safe_str(v.get("capRetailPrice")),
            "CAP average": safe_str(v.get("capAveragePrice")),
            "CAP below": safe_str(v.get("capUsedBelow")),
            "Autotrader Retail Value": safe_str(v.get("deltapointRetailPrice")),
            "Autotrader Trade Value": safe_str(v.get("deltapointTradePrice")),
            "Powered by": safe_str(v.get("assuredType")),
            "Inspection date": safe_str(inspection.get("date")).split("T")[0] if inspection.get("date") else "",
            "VIN": safe_str(v.get("vin")),
            "Exterior grade": "",
            "Interior grade": "",
            "Make": safe_str(v.get("make")),
            "Model": safe_str(v.get("model")),
            # "CC": round(float(v.get("engineSize", 0)) / 1000, 2) if v.get("engineSize") else "",
            "CC": round(float(v.get("engineSize", 0)) / 1000, 1) if v.get("engineSize") else "",
            "Engine number": "",
            "Body plan": safe_str(v.get("bodyType")),
            "Gearbox": safe_str(v.get("transmission")),
            "Paint type": "",
            "Colour Trim": safe_str(inspection.get("interiorTrim")),
            "Seats": safe_str(inspection.get("seats")),
            "Date of first reg": safe_str(v.get("firstRegistration")),
            "Keys": safe_str(inspection.get("keys")),
            "Inspection Report": safe_str(inspection.get("uncostedreporturl")),
            "Other Report": safe_str(v.get("assuredUrl")),
            "Tyres": ", ".join(
                f"{name}: {safe_str(value)}"
                for name, value in {
                    "NSF": tyres.get("nsf"),
                    "OSF": tyres.get("osf"),
                    "OSR": tyres.get("osr"),
                    "NSR": tyres.get("nsr"),
                    "Spare": tyres.get("spare"),
                }.items()
                if value not in ("", None)
            ),
            "Spare Tyre": safe_str(tyres.get("spare")),
            "Equipment": ", ".join(
                safe_str(opt.get("description"))
                for opt in safe_list(inspection.get("options"))
                if opt.get("description")
            ),
            "Damaged_images": ", ".join(
                safe_str(img)
                for d in safe_list(inspection.get("damage"))
                for img in safe_list(d.get("images"))
            ),
            "Damage_details": "; ".join(
                f"{safe_str(d.get('component'))} – {safe_str(d.get('damage'))}"
                for d in safe_list(inspection.get("damage"))
                if d.get("component") or d.get("damage")
            ),
            "Year": safe_str(v.get("firstRegistration"))[:4],
        }

        writer.writerow({h: row.get(h, "") for h in headers})

print(f"✅ CSV created successfully: {output_file}")


✅ CSV created successfully: barclay_renamed.csv


In [ ]:
#     # create a driver using chrome
#     driver = Chrome(service=service, options=options)
#     # run the driver through url
#     driver.get(path)
#     driver.maximize_window()
    

# # =====================================================================================
#     # handle cookies
# # =====================================================================================
#     try:
#         # time.sleep(1)
#         cookie = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/button[@id = "onetrust-accept-btn-handler"]')))
#         if cookie:
#             cookie.click()
#         else:
#             pass
#     except Exception as e:
#         print("No cookies tab found")


# # =====================================================================================
#     # get the login icon and click on it 
# # =====================================================================================
#     try:
#         # get the login button
#         login = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".acc_button_alt")))
#         if login:
#             driver.execute_script("arguments[0].scrollIntoView();", login)
#             login.click()
#         else:
#             pass

#     except Exception as e:
#         print(f"No login tab found")

# # =====================================================================================
#     # complete logiin info
# # =====================================================================================

#     # get the username tab
#     try:
#         provided_u_name = "sultanmirza0501@gmail.com"
#         user_name = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID,"Username")))   
#         user_name.send_keys(provided_u_name)
#     except Exception as e:
#         print(f"No username tab found")

#     # get password tab
#     try:
#         provided_pass = "Muhssan7865"
#         password = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "Password")))
#         password.send_keys(provided_pass)
#     except Exception as e:
#         print(f"No password tab found")

#     # get the login tab to enter the username and password
#     try:
#         # login_tab_css = "submit"
#         login_tab = driver.find_element(By.XPATH,  "//input[@type='submit' and @value='Login']")
#         driver.execute_script("arguments[0].scrollIntoView();", login_tab)
#         login_tab.click()
#         for i in range(2):
#             driver.back()
#     except Exception as e:
#         print(f"No login tab found")

# # =====================================================================================
#     # handle cookies
# # =====================================================================================
#     # try:
#     #     cookie = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler")))
#     #     if cookie:
#     #         cookie.click()
#     #     else:
#     #         pass
#     # except Exception as e:
#     #     print("No cookies tab found")

# # =====================================================================================
#     # get only number
# # =====================================================================================
#     try:
#         import time
#         time.sleep(2)
#         num_cars = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".d-md-flex span"))).text
#         cars_list = num_cars.split(" ")
#         cars =  int(cars_list[1]) # get only number
#         print(f"Number {cars} of cars found")
#     except Exception as e:
#         print(f"No number of cars found")

# # =====================================================================================
#     # get the car link and click on it to proceed to main car information page
# # =====================================================================================
#     try:
#         link = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vehicle-card-header")))
#         if link:
#         # now we have to click on the link to get the main car info page
#             driver.execute_script("arguments[0].scrollIntoView();", link)
#             link.click()
#         else:
#             pass
#     except Exception as e:
#         print(f"No link found")
# # =====================================================================================  


# # =====================================================================================
#     # loop around the number of images present
# # =====================================================================================

#     results = [] # to append the dict values later on
#     for i in range(cars):# we will make use of number of pages here
#         if not driver.window_handles:
#             print("Browser window closed. Stopping script.")
#             break

#     # ===================================================================================== 
#         # get the name of the car
#     # =====================================================================================
#         details = {}
        
#         try:
#             # Wait for car title element to appear (it can be <h2> or <h1>)
#             car_name_el = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((
#                     By.CSS_SELECTOR,
#                     "h2.vehicle-description-title, h1.vehicle-description, .ab-catelogue-vehicle-details-header h2"
#                 ))
#             )
#             car_name = car_name_el.text.strip()

#             if car_name:
#                 details["Title"] = car_name
#                 parts = car_name.split(" - ")
#                 if len(parts) > 1:
#                     description = parts[1].strip()
#                     words = description.split()
#                     index = next((i for i, w in enumerate(words) if any(ch.isdigit() for ch in w)), None)
#                     if index is not None and index + 1 < len(words):
#                         derivative = " ".join(words[index+1:])
#                         details["Derivative"] = derivative
#                     else:
#                         details["Derivative"] = "N/A"
#                 else:
#                     details["Derivative"] = "N/A"
#             else:
#                 details["Title"] = "No car name found"
#                 details["Derivative"] = "N/A"

#         except Exception as e:
#             print(f"❌ Error extracting car name: {e}")


        
#     # ===================================================================================== 
#         # extract the car images 
#     # =====================================================================================
#         try:
#             images=[]
#             # import time
#             time.sleep(2)
#             img_main =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.slider.animated"))) 
#             imgs = WebDriverWait(img_main, 3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.slide img")))
#             # driver.find_elements(By.CSS_SELECTOR, "li.slide img")

#             for img in imgs:
#                 images.append(img.get_attribute("src"))

#             imgs_str = ", ".join(images)
#             details['Images'] = imgs_str

#         except Exception as e:
#             print(f"No images found")

#     # =====================================================================================         
#         # get the lot time, date, auction and center fields
#     # =====================================================================================
#         try:
#             # container_selector = ".ab-catelogue-vehicle-details-auction-details"
#             container = driver.find_element(By.CSS_SELECTOR, ".ab-catelogue-vehicle-details-auction-details")

#             # Extract Auction Availability
#             auction_availability = container.find_element(By.TAG_NAME, "p").text
#             if auction_availability:
#                 details["Auction Availability"] = auction_availability
#             else:
#                 details["Auction Availability"] = "No auction availability found"

#             # Extract Lot, Time, and Date
#             lot_time_date = container.find_elements(By.CSS_SELECTOR, "h3 span")
#             if lot_time_date:
#                 lot = lot_time_date[0].text.replace("Lot: ", "")  # Removing 'Lot: ' prefix
#                 start_time = lot_time_date[1].text
#                 date = lot_time_date[2].text
#                 details['Lot'] = lot
#                 details['Start Time']= start_time
#                 details['Start Date'] = date
#             else:
#                 details['Lot'] = "na"
#                 details['Start Time']= "na"
#                 details['Start Date'] = "na"


#             # Extract Center
#             center = container.find_element(By.CSS_SELECTOR, "h3:nth-of-type(2) span").text.strip()
#             if center:
#                 details["Center"] = center.split("(")[0]
#                 details['Auction ID'] = center.split("(")[1].split(")")[0]
#             else:
#                 details['Center'] = "No center found"
#                 details['Auction ID'] = 'na'

#         except Exception as e:
#             print(f"Nothing found")
        
#     # =====================================================================================
#         # get the vehicle details
#     # =====================================================================================
#         try:
#             # click on vehicle tab
#             veh_tab = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.LINK_TEXT, "VEHICLE")))
#             if veh_tab:
#                 driver.execute_script("arguments[0].scrollIntoView();", veh_tab)
#                 veh_tab.click()

#                 base_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tab-content"]')))
                
#                 # Locate the table inside the card
#                 table_selector = ".tab-pane.active .table tbody tr"
#                 rows = base_card.find_elements(By.CSS_SELECTOR, table_selector)
                
#                 if rows:
#                     for row in rows:
#                         # Find all <td> elements in the row
#                         tds = row.find_elements(By.TAG_NAME, "td")
#                         if len(tds) == 2:  # Ensure there are exactly two <td> elements
#                             label = tds[0].text.strip()

#                             # Special case for "Mileage warranted"
#                             if label == "Mileage warranted" or label == "V5":
#                                 try:
#                                     path_element = tds[1].find_element(By.TAG_NAME, "path")
#                                     value = path_element.get_attribute("d")  # Get the 'd' attribute
#                                 except Exception as e:
#                                     value = "Path not found"
#                             else:
#                                 value = tds[1].text.strip()

#                             details[label] = value
#                 else:
#                     print("No rows found")
                
#             else:
#                 print("No vehicle tab found")
#         except Exception as e:
#             print(f"Novehcile info found")

#     # =====================================================================================
#         # get the valuations
#     # =====================================================================================
#         try:

#             valuation = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.LINK_TEXT, "VALUATIONS")))
#             if valuation:
#                 driver.execute_script("arguments[0].scrollIntoView();", valuation)
#                 valuation.click()
#                 import time
#                 time.sleep(1)

#                 val_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "tab-content")))
#                 if val_card:
#                     val_rows = val_card.find_elements(By.CSS_SELECTOR, ".tab-pane.active .table tbody tr")
#                     if val_rows:         
#                         for val_row in val_rows:
#                             # Find all <td> elements in the row
#                             val_tds = val_row.find_elements(By.TAG_NAME, "td")
#                             if len(tds) == 2:  # Ensure there are exactly two <td> elements
#                                 val_label = val_tds[0].text.strip()
#                                 val_value = val_tds[1].text.strip()
#                                 details[val_label] = val_value
#                     else:
#                         print("No value rows found inside valaution card")
#                 else:
#                     print("No valuation card found")
#             else:
#                 print("No valaution button found")

#         except Exception as e:
#             print(f"No valaution button found")
        
#     # =====================================================================================
#         # get the INSPECTION REPORT
#     # =====================================================================================
#         try:

#             inspection = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.LINK_TEXT, "INSPECTION REPORT")))
#             if inspection:
#                 driver.execute_script("arguments[0].scrollIntoView();", inspection)
#                 inspection.click()
#                 import time
#                 time.sleep(1)
            
#                 # Now extract the fields inside the inspection report
#                 inspec_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "tab-content")))
#                 if inspec_card:
#                     insp_rows = inspec_card.find_elements(By.CSS_SELECTOR, ".tab-pane.active .table tbody tr")
#                     if insp_rows:         
#                         for insp_row in insp_rows:
#                             # Find all <td> elements in the row
#                             insp_tds = insp_row.find_elements(By.TAG_NAME, "td")
#                             if len(insp_tds) == 2:  # Ensure there are exactly two <td> elements
#                                 insp_label = insp_tds[0].text.strip()
#                                 insp_value = insp_tds[1].text.strip()
#                                 details[insp_label] = insp_value
#                     else:
#                         print("No value rows found inside valaution card")
#                 else:
#                     print("No valuation card found")
            
#             # =====================================================================================
#                 # get the icon image link at the side of the name of the car
#             # =====================================================================================
#                 try:
#                     # inspec_rep_css = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Download the inspection report']")))
#                     inspec_rep_css = driver.find_element(By.XPATH, "//a[text()='Download the inspection report']")
#                     if inspec_rep_css:
#                         inspec_link = inspec_rep_css.get_attribute('href')
#                         details["Inspection Report"] =  inspec_link
#                     else:
#                         details["Inspection Report"] = "No inspection report found"
#                 except Exception as e:
#                     print("No car inspection report tab found")

#             # ===================================================================================== 
#                 # get the assured report link at the side of the name of the car
#             # =====================================================================================
#                 try:
#                     icon_image_link = inspec_card.find_element(By.XPATH, "//a[contains(@href, '.pdf')]")

#                     if icon_image_link:
#                         icon_href = icon_image_link.get_attribute("href")
#                         details['Other Report'] = icon_href
#                     else:
#                         details['Other Report'] = "N/A"
#                 except Exception as e:
#                     print(f"No icon image tab found")

                
#                 # tyres
#                 try:
#                     tyre_main = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/h3[contains(text(), "Tyres")]//following::table[1]')))
#                     if tyre_main:
#                         tyre_trs = WebDriverWait(tyre_main, 3).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
#                         if tyre_trs:
#                             for tyre_tr in tyre_trs:
#                                 tyre_lbl = tyre_tr.find_elements(By.TAG_NAME, "td")
#                                 if tyre_lbl:
#                                     details[tyre_lbl[0].text.strip()] = [tyre_lbl[1].text.strip(), tyre_lbl[2].text.strip()]
#                         else:
#                             print("No interior")
#                 except:
#                     print(f"No tyre data")
#             else:
#                 print("No inspection button found")
#         except Exception as e:
#             print(f"No inspection button found")
        
#     # =====================================================================================
#         # get the SPECIFICATION
#     # =====================================================================================
#         try:
#             # get the button and then click on it
#             spec_text = "SPECIFICATION"
#             specs = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.LINK_TEXT, spec_text)))
#             if specs:
#                 driver.execute_script("arguments[0].scrollIntoView();", specs)
#                 specs.click()
#                 import time
#                 time.sleep(1)

#                 # now extract the fields inside valautions
#                 spec_card_class = "tab-content"
#                 spec_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, spec_card_class)))
                
#                 # Locate the table inside the card
#                 spec_table_selector = ".tab-pane.active .table tbody tr"
#                 spec_rows = spec_card.find_elements(By.CSS_SELECTOR, spec_table_selector)
                
#                 td_texts = [td.text for td in spec_rows if td.text.strip() != ""]
#                 details['Equipment']=td_texts
#             else:
#                 print("No specs button found")
#         except Exception as e:
#             print(f"No specification button found")


#     # =====================================================================================
#         # Extract the image source URL
#     # =====================================================================================
#         try:
#             damage_text = "DAMAGE REPORT"
#             damages = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.LINK_TEXT, damage_text)))
#             if damages:
#                 driver.execute_script("arguments[0].scrollIntoView();", damages)
#                 damages.click()
#                 import time
#                 time.sleep(1)

#                 # Initialize list for storing damaged image URLs and key-value pairs
#                 damaged_list = []
#                 details_list = []

#                 # Extract all image elements with the class 'magnifier-image'
#                 img_elements = driver.find_elements(By.CLASS_NAME, 'magnifier-image')
#                 for img in img_elements:
#                     damaged_list.append(img.get_attribute("src"))
                
#                 # Assuming there's only one image to focus on for now
#                 damaged_urls = ", ".join(damaged_list)
#                 details["Damaged_images"] = damaged_urls

#                 # Extract key-value pairs from the damage table
#                 damage_rows = driver.find_elements(By.CSS_SELECTOR, '.tab-pane.active .table tbody tr')

#                 for damage_row in damage_rows:
#                     # Get the first and second <td> elements
#                     columns = damage_row.find_elements(By.TAG_NAME, 'td')
#                     if len(columns) >= 2:  # Ensure there are exactly two columns
#                         column_name = columns[0].text.strip()
#                         column_value = columns[1].text.strip()
#                         details_list.append({column_name: column_value})

#                 details["Damage_details"] = details_list  # Add to the 'details' dictionary
#             else:
#                 print("No damage button found")
#         except Exception as e:
#             print(f"No damage button found")
#     # =====================================================================================
#     # =====================================================================================

#         # append the in the results 
#         results.append(details)

#     # =====================================================================================
#         # get the next button
#     # =====================================================================================
#         try:
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.vehicle-details-btn-next")
#             if next_button:
#                 # get the link
#                 # car_page_link = next_button.get_attribute("href")
#                 # details['Car_link']= car_page_link
#                 next_button.click()
#             else:
#                 # details['Car_link'] = "N/A"
                
#                 break
#         except Exception as e:
#             print("No next button found hence no more cars are available")

# # =====================================================================================
#     # create a dataframe 
#     df = pd.DataFrame.from_dict(results)
#     df.to_csv("Barclay_data.csv", index=False)

#     driver.quit()

# # call the function
# path = "https://catalogue.astonbarclay.net/vehicle-search/?search=true&sites=1&sales=17524"
# scrape(path)



In [ ]:
# df=pd.read_csv("Barclay_data.csv")
# df.rename(columns={"REG":"Reg", "Fuel":"Fuel Type", "MOT expires":"MOT Expiry Date", "Service":"Service History", "Notes":"Service Notes", "NAMA grade":"Grade", "CAP clean price":"CAP clean", "CAP retail price":"CAP retail", "CAP average price":"CAP average", "CAP below price":"CAP below", "Current retail value":"Autotrader Retail Value", "Current trade value":"Autotrader Trade Value", "NSF":"Tyre NSF", "OSF":"Tyre OSF", "NSR":"Tyre NSR", "OSR":"Tyre OSR", "Spare":"Spare Tyre", "Internal grade":"Interior grade", "D.O.R.":"D.O.R", "Int. colour trim":"Colour Trim", "Seller":"Vendor", "CN":"VIN", "Engine size":"CC"}, inplace=True)
# df['Year'] = df['D.O.R'].apply(lambda x : x.split("/")[-1] if isinstance(x, str) and "/" in x else "na")
# df['Last Service'] = df['Service History'].apply(lambda x : f"02/" + x.split(" ")[-1].split("-")[0] if isinstance(x, str) and " "in x and "-" in x else 'na')
# df['Last Service Mileage'] = df['Service History'].apply(lambda x : x.split(" ")[-1].split(",")[0].split("-")[-1] if isinstance(x, str) and " " in x and "," in x and "-" in x else 'na')

# df['CC'] = df['CC'].replace('-', 0).astype(int)

# df['CC'] = df['CC'].apply(lambda x : float(x/1000) if x else "na")

# df.to_csv("barclay_renamed.csv", index=False)
# os.remove("Barclay_data.csv")

### images and reports

In [ ]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from Aston Barclays"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [ ]:
from urllib.parse import urlparse, urljoin

df = pd.read_csv("barclay_renamed.csv")

reg_img = df[['Reg', "Images", "Damaged_images"]]
reports = df[['Reg', "Inspection Report","Other Report"]]


def download_images(data, main_folder="Images"):

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        
     
        image_urls = row["Images"].split(", ") if pd.notna(row["Images"]) else []
        damaged_urls = row["Damaged_images"].split(", ") if pd.notna(row["Damaged_images"]) else []
        
   
        reg_folder = os.path.join(main_folder, reg_no) 
        os.makedirs(reg_folder, exist_ok=True)  
       
        images_folder = os.path.join(reg_folder, "Original_images") 
        os.makedirs(images_folder, exist_ok=True)  

       
        damaged_folder = os.path.join(reg_folder, "Damaged_images")  
        os.makedirs(damaged_folder, exist_ok=True)  

   
        for idx, url in enumerate(image_urls):
            url = url.strip()  
            if not url.startswith(("http://", "https://")):  
                url = urljoin("https://", url)  

        
            parsed_url = urlparse(url)

    
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue
            

            try:
     
                response = requests.get(url, stream=True)
                response.raise_for_status() 

                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

         
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"  
                
              
                full_file_name = os.path.join(images_folder, f"{reg_no}_{idx + 1}.jpg")
                
          
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

            
                add_watermark_to_image(full_file_name)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

   
        for idx, url in enumerate(damaged_urls):
            url = url.strip()  
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)  

         
            parsed_url = urlparse(url)

     
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue
            
  
            try:
          
                response = requests.get(url, stream=True)
                response.raise_for_status()  

    
                file_name = os.path.basename(parsed_url.path) or f"damaged_{idx + 1}.jpg"

        
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"damaged_{idx + 1}.jpg"  
                
          
                full_file_name = os.path.join(damaged_folder, f"{reg_no}_{idx + 1}.jpg")
                
         
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

          
                add_watermark_to_image(full_file_name)

                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")


def download_reports(data, main_folder="Reports"):
    os.makedirs(main_folder, exist_ok=True)

    for i, row in data.iterrows():
        reg_no = str(row["Reg"]).strip()
        inspection_url = row["Inspection Report"]
        other_url = row["Other Report"]

        # Folder for this vehicle
        vehicle_folder = os.path.join(main_folder, reg_no)
        os.makedirs(vehicle_folder, exist_ok=True)

        # Subfolders
        inspection_folder = os.path.join(vehicle_folder, "Inspection Report")
        other_folder = os.path.join(vehicle_folder, "Other Report")
        os.makedirs(inspection_folder, exist_ok=True)
        os.makedirs(other_folder, exist_ok=True)

        # --- Helper Function for Download ---
        def download_file(url, dest_folder, file_label):
            if not url or pd.isna(url):
                print(f"❌ Missing {file_label} for {reg_no}")
                return
            
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)

            parsed = urlparse(url)
            if not parsed.scheme or not parsed.netloc:
                print(f"⚠️ Invalid URL skipped for {reg_no}: {url}")
                return

            try:
                response = requests.get(url, stream=True, timeout=15)
                response.raise_for_status()
                file_path = os.path.join(dest_folder, f"{reg_no}.pdf")

                with open(file_path, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                print(f"✅ Downloaded {file_label} for {reg_no}")
            except Exception as e:
                print(f"❌ Failed {file_label} for {reg_no}: {e}")

        # Download both reports
        download_file(inspection_url, inspection_folder, "Inspection Report")
        download_file(other_url, other_folder, "Other Report")

    print("🎉 All reports processed.")


def start_funcs():
    thread1 = threading.Thread(target=download_images, args=(reg_img,))
    thread2 = threading.Thread(target=download_reports, args=(reports,))
    
    # start threads 
    thread1.start()
    thread2.start()

    # join threads
    thread1.join()
    thread2.join()
    
if __name__ == "__main__":
    start_funcs() 

Watermark added to Images\MF14KWP\Original_images\MF14KWP_1.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_1.jpg
Watermark added to Images\MF14KWP\Original_images\MF14KWP_2.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_2.jpg
✅ Downloaded Inspection Report for MF14KWP
Watermark added to Images\MF14KWP\Original_images\MF14KWP_3.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_3.jpg
Watermark added to Images\MF14KWP\Original_images\MF14KWP_4.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_4.jpg
✅ Downloaded Other Report for MF14KWP
Watermark added to Images\MF14KWP\Original_images\MF14KWP_5.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_5.jpg
Watermark added to Images\MF14KWP\Original_images\MF14KWP_6.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_6.jpg
✅ Downloaded Inspection Report for SD67YSS
Watermark added to Images\MF14KWP\Original_images\MF14KWP_7.jpg
Downloaded: Images\MF14KWP\Original_images\MF14KWP_7.jpg
Watermark added to Images\MF1